In [1]:
import pandas as pd
import numpy as np
from readFile import split_into_values, toRPdata

In [2]:
# columns 와 value는 사용자 입력
df = pd.read_csv('resources/AXISX_resample.csv')
#df = pd.read_csv('resources/CLAMP_resample.csv')
value = ['value']
#df = pd.read_csv('resources/Dataset1.csv')
columns = ['chip', 'wire', 'segment']
value = ['value']

In [3]:
df = df.loc[:, columns + value] #('chip', 'wire', 'value')는 사용자 입력
size = 96 # 모델에 따라 변경 28, 96
result = split_into_values(df, columns)

In [7]:
# dataframe -> list
result_list = result.values.tolist()

In [8]:
from sklearn.preprocessing import MinMaxScaler

def MinMax(data):
    MMS = MinMaxScaler().fit(data)
    scaled = MMS.transform(data)
    return scaled

In [9]:
# result_list transpose
result_T = [list(x) for x in zip(*result_list)]

In [10]:
# minmax 정규화
result_scaled = MinMax(result_T)

In [11]:
# 다시 result transpose 해서 원래대로
result_scaled = [list(x) for x in zip(*result_scaled)]

In [12]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [13]:
# 2. 시계열 셋 크기 변경
result_ = TimeSeriesResampler(sz=size).fit_transform(result)

In [14]:
data = result_.reshape(result_.shape[0], 1, size)
X = toRPdata(data, threshold='point', percentage=30) # rp 1
# X = toRPdata(data) # rp 2

In [15]:
X_scaled = np.expand_dims(X, axis=3)
X_scaled.shape

(1330, 96, 96, 1)

In [16]:
from keras.models import load_model

In [17]:
model = load_model('rp1_AXIXS_96_Autoencoder_128_3e-05_weight.h5')

In [18]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 96, 96, 32)        320       
_________________________________________________________________
activation_35 (Activation)   (None, 96, 96, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 48, 48, 16)        4624      
_________________________________________________________________
activation_36 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 24, 24, 2)        

In [19]:
from keras import backend as K

compressed_layer = 8
get_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
compressed = get_layer_output([X_scaled])[0]
feature = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])

print(feature)
print(feature.shape)

[[1.235375   0.0543054  1.2296575  ... 0.         1.0501287  0.        ]
 [1.179232   0.         1.1642938  ... 0.         1.0493768  0.        ]
 [1.15436    0.         0.9993946  ... 0.         1.1852528  0.        ]
 ...
 [0.76522005 0.         0.56581473 ... 0.         0.9489112  0.        ]
 [1.0703907  0.05048279 0.91729414 ... 0.         1.3885974  0.        ]
 [1.0146557  0.03062279 0.6021828  ... 0.         1.3548934  0.        ]]
(1330, 288)


In [21]:
import numpy

numpy.save('./compressed', compressed) # x_save.npy
numpy.save('./feature', feature) # x_save.npy
numpy.save('./X_scaled', X_scaled) # x_save.npy